In [1]:
# =============================================================================
# Assignment: Final Project - Phase 2
# Filename: final_project_2.py
# Description: Final Project
# Date: 04/19/2020
# Author: Tarini Dash
# =============================================================================

import pandas as pd
import numpy as np

# NAME YOUR DATAFRAMES.  I KNOW DF IS COMMON BUT IT'S ONLY BECAUSE PEOPLE ARE USING IT GENERICALLY WHEN DISCUSSING
# PANDAS.  YOU WILL HAVE MORE THAN ONE DATAFRAME IN PROJECTS.

# LET ME SHOW YOU A FEW OTHER WAYS TO DO THIS WITHOUT USING DROP, BUT IN ORDER TO DO SO I NEED TO SHOW YOU SOME OTHER
# TRICKS!

df = df.drop(columns=['Scn', 'Class'])

# FIRST, YOU DON'T NEED THE COLUMN NAMES:

cols = ["Scn", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "Class"] 
df = pd.read_csv('breast-cancer-wisconsin.data', na_values = '?', names = col)

# THEY WILL BE INFERRED BY READ_CS. IF YOU ALREADY HAVE A HEADER THEN YOU CAN JUST DO THIS (ALSO I DON'T THINK YOU 
# NEED TO REPLACE NaN's WITH '?', I KNOW THAT VEL DOES):

data = pd.read_csv('breast-cancer-wisconsin.data')
print(data.columns) # ["Scn", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "Class"]

# SEE?  ALREADY INCLUDED!  AND, YOU CAN ALWAYS SEE THE HEADER USING THE .columns PROPERTY OF THE DATAFRAME. OF 
# COURSE YOU CAN CHANGE THE COLUMNS EXCLUDE THEM ENTIRELY IN read_csv.  TAKE A LOOK AT read_csv API.   

# THE columns PROPERTY IS LIKE A LIST, IT'S NOT, BUT YOU CAN DO THIS WITH IT TO REMOVE COLUMNS SO LONG AS THEY ARE
# CONTINUOUS:

data = data[data.columns[1:-1]]

# OR

data = data[data.columns[:4]]

# BUT IF YOU MAKE THEM INTO A LIST, YOU CAN ALSO REORDER AND DROP COLUMNS IN THE MIDDLE LIKE SO:

cols = list(data.columns)
data = data[cols[6:] + cols[:6]]
print(data.columns) [ "A7", "A8", "A9", "A10", "Class", "Scn", "A2", "A3", "A4", "A5", "A6"]

# IN ORDER TO AVOID HAVING TO DO THAT FOR EACH FUNCTION, I DID IT ONCE AT THE BEGINNING AS A SEPPERATE DATASET YOU
# COULD LOAD IT AS original AND THEN SET A DATAFRAME WITHOUT THE FIRST AND LAST COLUMNS TO DATA OR SOMETHING.  YOU
# THEN, I JUST WORKED WITH THAT DATA FRAME, AND ADDED data['Predicted'] to original['Predicted'] AT THE PART I NEEDED
# TO, BUT THEN IF I HAD TO DO ANOTHER LOOP, I WOULD SWITCH BACK TO THE data DATAFRAME AND DO IT ALL OVER AGAIN.

def initial(df):
    df = df.drop(columns=['Scn', 'Class']) # DROP THESE ONCE AT THE BEGINNING AND CALL THE DATASET 'TRAIN'
    # ref - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html
    return df.sample(2)


def get_cluster_value(df, index, mu_2, mu_4):
    df = df.drop(columns=['Scn', 'Class'])
    
    # GOOD, WE'LL RETURN TO THE INDEX IN A SECOND, BUT START BELOW.
    
    dist_from_centroid_2 = np.sqrt(np.square(df.loc[index] - mu_2).sum())
    dist_from_centroid_4 = np.sqrt(np.square(df.loc[index] - mu_4).sum())
    
    # YOU DON'T HAVE TO DO BELOW.  YOU GOTTA START THINKING IN TERMS OF SERIES AND FRAMES WHICH ACT LIKE VECTORS 
    # A) WHAT YOU HAVE ABOVE IS ALREDY COMPUTING A SERIES (VECTOR) BUT HERE YOU ARE ONLY RETURNING ONE VALUE.  WHEN 
    # YOU WRITE YOUR IF, WHAT YOUR ASKING IS IF THE *ENTIRE* SERIES (VECTOR) dist_from_centroid_2 IS LESS THAN THE 
    # *ENTIRE* SERIES (VECTOR) dist_from_centroid_4.  IF IT IS, YOUR RETURNING THE SINGLE DIGIT (SCALER) 2 OR 4 
    # DEPENDING.  THIS IS WHAT'S CAUSING YOU TO HAVE TO LOOP IN YOUR ASSIGN CLUSTER FUNCTION
    
# %%%%% REMOVE %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%   
#     if(dist_from_centroid_2 < dist_from_centroid_4):
#         return 2
#     if(dist_from_centroid_4 < dist_from_centroid_2):
#         return 4
#%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

    # INSTEAD, YOU CAN DO ONE OF A BUNCH OF THIGNS:
    
    # OPTION 1: 4 LINES
    df['predicted'] = dist_2 < dist_4
    df['predicted'].head()
    # TRUE
    # FALSE
    # TRUE
    # FALSE
    # TRUE
    
    df.loc[df['predicted'] == True, 'predicted'] = 2
    df.loc[df['predicted'] == False, 'predicted'] = 4
    df.head()
    # 2
    # 4
    # 2
    # 4
    # 2
    
    # OPTION 2: EVEN BETTER, THREE LINES!
    
    df['predicted'] = dist_2 < dist_4
    df['predicted'] = df['predicted'].replace({True: 2, False: 4})
    
    # OPTION 3: EVEN MORE BETTER, ONE LINE!
    
    df['predicted'] = (dist_2 < dist_4).replace({True: class_a, False: class_b})
    
    # THIS IS ALL PERMITTED BECAUSE I'M USING SERIES dist_2 < dist_4 RETURNS A SERIES OF TRUE'S AND FALSE'S THEN
    # I'M SAYING TO PUT A 2 IN THE SERIES WHEREVER A TRUE IS AND A 4 IN THE SERIES WHEREVER A FALSE IS.  THEN I'M
    # STICKING THE ENTIRE SERIES INTO THE DATAFRAME AS A COLUMN (WHICH IS A SERIES).
    
    # SO IF YOU DON'T NEED A LOOP ANYMORE, THAT MEANS YOU DON'T NEED AN INDEX!  THEREFORE YOUR WHOLE FUNCTION 
    # BECOMES: 
    
    def get_cluster_value(df, mu_2, mu_4):
        df = df.drop(columns=['Scn', 'Class'])

        # NO INDEX BECAUSE IT WILL DO WHAT YOU HAVE BELOW TO THE *ENTIRE* DATAFRAME AUTOMATICALLY!

        dist_from_centroid_2 = np.sqrt(np.square(df - mu_2).sum())
        dist_from_centroid_4 = np.sqrt(np.square(df - mu_4).sum())
        
        # IT'S YOUR CHOICE FROM ABOVE BUT WE CAN GO MIDWAY SO THAT PEOPLE CAN SEE THE STEPS
        
        # IS RETURNS A *SERIES* THAT ARE BOOLEANS WHERE THE CONDITION IS TRUE VS. FALSE
        cluster_assign =  dist_from_centroid_2 < dist_from_centroid_4

        # REPLACES THE TRUE'S AND FALSE'S WITH 2's and 4's
        cluster_assign = cluster_assign.replace({True: 2, False: 4})
        
        # ADDS THE SERIES TO THE DATAFRAME AS A NEW COLUMN
        df["Predicted_Class"] = cluster_assign

    
    # BY THE WAY.  IN YOUR CURRENT ASSIGN CLUSTER FUNCTION, I THINK YOU ARE TRYING TO APPEND A SINGLE CELL IN EACH
    # COLUMN TOGETHER TO MAKE A ROW?  NO NEED.  YOU SHOULDN'T BE LOOPING ANYWAY, BUT IF YOU WERE, YOU CAN ASK FOR
    # df.iloc[n] WHERE n IS A NUMBER IN THE INDEX. YOU CAN REMEMBER THAT iloc GETS YOU ROW'S BECUASE THE i IS FOR
    # INDEX.  YOU'RE SAYING, GET ME THE ROW AT INDEX n.  ASIDE: YOU CAN ALSO GET THE NUMBER OF AN INDEX BY DOING
    # df.index[n].  BOTH WORK WITH THINGS LIKE THIS df.iloc[4:10] AS WELL
    
    # SO THINK ABOUT THIS AND TRY TO START THINKING IN THIS WAY.  SEE HOW IT WOULD CHANGE YOUR APPROACH WERE YOU TO
    # HAVE THE TIME TO DO IT.
    
    # SMALL NOTE: DON'T FfORGET TO ADD A CELL FOR MAIN()



NameError: name 'df' is not defined

In [2]:
    
    


    
def recompute(cluster_2,cluster_4):
    cluster_2 = cluster_2.drop(columns=['Scn', 'Class','predicted_class'])
    cluster_4 = cluster_4.drop(columns=['Scn', 'Class','predicted_class'])
    return cluster_2.mean(axis = 0), cluster_4.mean(axis = 0) # .mean() is already zero by default




def assign_cluster(df,mu_2,mu_4):
    col1 = ["Scn", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "Class","predicted_class"]
    predicted_clusters = pd.DataFrame()
    for i in range(len(df)):
#         data = pd.Series([df["Scn"].loc[i],
#                           df["A2"].loc[i],
#                           df["A3"].loc[i],
#                           df["A4"].loc[i],
#                           df["A5"].loc[i],
#                           df["A6"].loc[i],
#                           df["A7"].loc[i],
#                           df["A8"].loc[i],
#                           df["A9"].loc[i],
#                           df["A10"].loc[i],
#                           df["Class"].loc[i]
#                           ,get_cluster_value(df,i,mu_2,mu_4)], index = col1)
#         print(data)
        
        predicted_clusters = predicted_clusters.append(pd.Series([df["Scn"].loc[i],
                                                                  df["A2"].loc[i],
                                                                  df["A3"].loc[i],
                                                                  df["A4"].loc[i],
                                                                  df["A5"].loc[i],
                                                                  df["A6"].loc[i],
                                                                  df["A7"].loc[i],
                                                                  df["A8"].loc[i],
                                                                  df["A9"].loc[i],
                                                                  df["A10"].loc[i],
                                                                  df["Class"].loc[i]
                                                                  ,get_cluster_value(df,i,mu_2,mu_4)], index = col1) ,ignore_index=True)
        
    return predicted_clusters


    
def loop_through(df,mu_2,mu_4):
    for i in range(1,51):
#         print("mu_2 :", mu_2)
#         print("mu_4 :", mu_4)
        predicted_clusters = assign_cluster(df,mu_2,mu_4)
            
        cluster_2 = predicted_clusters.loc[predicted_clusters['predicted_class'] == 2]
        cluster_4 = predicted_clusters.loc[predicted_clusters['predicted_class'] == 4]
    
        new_mu_2, new_mu_4 = recompute(cluster_2,cluster_4)
        if(np.array_equal(new_mu_2, mu_2) & np.array_equal(new_mu_4, mu_4)):
#             print("centroids matched with previous centroids. No change in centroids")
            return i, mu_2, mu_4, predicted_clusters
            break;
        else:
            mu_2 = new_mu_2
            mu_4 = new_mu_4
    return i, mu_2, mu_4, predicted_clusters 
            


def main():
    #ref - https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.set_option.html
    pd.set_option('precision', 0)
    
    col = ["Scn", "A2", "A3", "A4", "A5", "A6", "A7", "A8", "A9", "A10", "Class"]
    
    # read from file
    df = pd.read_csv('breast-cancer-wisconsin.data', na_values = '?', names = col)
 
    # replace missing value with mean
    df = df.fillna(df.mean()['A7'])
    
    df_sample = initial(df)
    
    count = 2
    for index, row in df_sample.iterrows():
        print("Randomly selected row ",index, "for centroid mu_"+str(count), end="\n\n")
        print("Initial centroid mu_"+str(count)+":")
        print(row,end="\n\n")
        count += 2
    
    mu_2 , mu_4 = df_sample.values[0].astype(int), df_sample.values[1].astype(int)
    
    i,final_mu_2,final_mu_4, predicted_clusters = loop_through(df,mu_2,mu_4)

    print("Program ended after ",i, " iterations.",end="\n\n")
    print("Final centroid mu_2:\n",final_mu_2,end="\n\n")
    print("Final centroid mu_4:\n",final_mu_4,end="\n\n")
    print("Final cluster assignment:",end="\n\n")
    predicted_clusters = predicted_clusters[['Scn', 'Class', 'predicted_class']]
    print(predicted_clusters,end="\n\n")
#     print(predicted_clusters.head(20),end="\n\n")
    
    cluster_2 = predicted_clusters.loc[predicted_clusters['predicted_class'] == 2]
    cluster_4 = predicted_clusters.loc[predicted_clusters['predicted_class'] == 4]
    print("Data points in Predicted Class 2: ",cluster_2.shape[0])
    print("Data points in Predicted Class 4: ",cluster_4.shape[0],end="\n\n")
    
    error_cluster_2 = cluster_2.loc[cluster_2['Class']== 4]
    error_cluster_4 = cluster_4.loc[cluster_4['Class']== 2]
    print("Error data points, Predicted Class 2:\n",error_cluster_2,end="\n\n")
    print("Error data points, Predicted Class 4:\n",error_cluster_4,end="\n\n")
     
    print("Number of all data points:   ", predicted_clusters.shape[0],end="\n\n")
    
    print("Number of error data points: ", error_cluster_2.shape[0] + error_cluster_4.shape[0],end="\n\n")
    
    print("Error rate for class 2:      ", round((error_cluster_2.shape[0]/predicted_clusters.shape[0])*100,1),"%")
    print("Error rate for class 4:      ", round((error_cluster_4.shape[0]/predicted_clusters.shape[0])*100,1),"%")
    print("Total error rate:            ", round(((error_cluster_2.shape[0] + error_cluster_4.shape[0])/predicted_clusters.shape[0])*100,1),"%")
    
    
#invoke main method
if __name__ == "__main__":
    main()

NameError: name 'initial' is not defined